# Introdução

Nesse notebook, vamos explorar algumas modificações de GANs que melhoram a GAN original em alguns aspectos. Em duas dessas abordagens (DCGAN e WGAN-GP), o objetivo era melhorar a estabilidade do treinamento, e em uma delas (Pix2Pix) o objetivo era extender as GANs para uma tarefa ligeiramente diferente, de "tradução" de imagem para imagem.

In [ ]:
!rm -r PyTorch-GAN
!git clone https://github.com/eriklindernoren/PyTorch-GAN
%cd /content/PyTorch-GAN/
!sudo pip3 install -r requirements.txt

# DCGANs

A primeira arquitetura que vamos ver é a [DCGAN](https://arxiv.org/abs/1511.06434), que foi um trabalho seminal de 2016 que trouxe alguns padrões de arquitetura e de treinamento para as GANs. Antes desse trabalho, a geração de imagens de resolução média ainda era um desafio e com essa abordagem se atraiu ainda mais interesse de pesquisadores para as GANs. A arquitetura do Gerador é a seguinte:

![DCGAN Architecture](https://miro.medium.com/max/700/1*KvMnRfb76DponICrHIbSdg.png)



A cartilha para treinamento das GANs proposta pelo artigo é a seguinte:

![DCGAN guideline](https://i.imgur.com/08EVNUb.png)



Imagens geradas ao longo do treinamneto das DCGANs:

![DCGANs](https://github.com/eriklindernoren/PyTorch-GAN/raw/master/assets/dcgan.gif)

In [ ]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from matplotlib import pyplot as plt
%matplotlib inline

os.makedirs("images", exist_ok=True)

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

Primeiramente vamos acessar o diretório de implementação da DCGAN, do repositório que estamos usando.

In [ ]:
%cd /content/PyTorch-GAN/implementations/dcgan/

Declarando hiperparâmetros para usar no treinamento desse modelo:

In [ ]:
class Opt:
    epoch = 1
    n_epochs = 10
    batch_size = 128
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    n_cpu = 4
    latent_dim = 100
    img_size = 64
    channels = 1
    sample_interval = 200

opt = Opt()

Agora vamos declarar a arquitetura do Gerador e do Discriminador:

In [ ]:
cuda = True if torch.cuda.is_available() else False

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

Função de perda:

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

In [ ]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


Carregando dataloader para o MNIST:

In [ ]:
# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

Definindo otimizadores:

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Definindo o processo de treinamento:

In [ ]:
## definindo uma função para declaração de tensores, dependendo se estamos na GPU ou não
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            
            fig, ax = plt.subplots(1, 10, figsize=(20, 2))
            
            for b in range(10):
                
                ax[b].imshow(gen_imgs.data[b].detach().cpu().numpy().squeeze())
                ax[b].set_yticks([])
                ax[b].set_xticks([])
            
            plt.show()

# Pix2Pix

O [Pix2Pix](https://phillipi.github.io/pix2pix/) trouxe para as GANs uma tarefa bastante diferente da GAN tradicional. A partir da GAN condicional, os autores desse trabalho observaram que podemos condicionar o Gerador usando (praticamente) qualquer informação que pode ser fornecida como entrada de uma rede neural. No Pix2Pix, então foi proposta a idéia de condicionar a geração em uma imagem, com o objetivo de gerar uma outra imagem. Por isso, esse processo ficou chamado de tradução imagem-para-imagem (*image-to-image translation*). No artigo original, eles exemplificam vários domínios onde essa arquitetura pode ser usada, e criaram uma ferramenta para que a comunidade em geral pudesse usar a arquitetura de várias formas diferentes. O treinamento dessa arquitetura é feita da seguinte forma:

![Pix2Pix D and G](https://camo.githubusercontent.com/e8c023b62678aa244f1a474bf643c66c45ef0feb/687474703a2f2f6572696b6c696e6465726e6f72656e2e73652f696d616765732f706978327069785f6172636869746563747572652e706e67)

Exemplos de imagens geradas por essa arquitetura (no artigo original, resultados melhores são reportados):

![Pix2Pix Examples](https://github.com/eriklindernoren/PyTorch-GAN/raw/master/assets/pix2pix.png)

Fazendo alguns procedimentos necessários:

- Download dos dados usados para esse experimento (Facades Dataset)
- Migrando para o diretório (importado do github) que contém as implementações das arquiteturas

In [ ]:
## download dos dados Facades
%cd /content/PyTorch-GAN/
%cd data/
!bash download_pix2pix_dataset.sh facades > out.log

## migrando para diretório com implementações dos modelos
%cd /content/PyTorch-GAN/implementations/pix2pix/

In [ ]:
import argparse
import os
import numpy as np
import math
import itertools
import time
import datetime
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

## importando modelos e classes necessárias para carregar os dados (classe
## personalizada para ler do disco o dataset Facades)
from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

from matplotlib import pyplot as plt
%matplotlib inline

Hiperparâmetros para treinamento desse modelo:

In [ ]:
class Opt:
    epoch = 1
    n_epochs = 20
    dataset_name = 'facades'
    batch_size = 1
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    decay_epoch = 100
    n_cpu = 4
    img_height = 256
    img_width = 256
    channels = 3
    sample_interval = 200
    checkpoint_interval = -1

opt = Opt()
print(opt)

In [ ]:
cuda = True if torch.cuda.is_available() else False

Funções de perda. No caso dessa tarefa, queremos que o Gerador seja capaz de criar imagens tanto que pareçam reais, quanto que seguem a estrutura global da imagem real "ground truth". Portanto, temos duas funções de perda:

- Uma loss padrão das GANs de separação entre imagens reais e falsas `MSELoss`.
- Outra que penaliza o quanto a imagem gerada por G se difere da imagem real esperada `L1Loss`. Essa loss é dita "pixelwise" porque ela penaliza a diferença de valores de cada pixel da imagem gerada com a imagem original.

A segunda loss é necessária porque, se ela não existisse, o Gerador não seria penalizado por gerar uma imagem que parece real mas que não segue a estrutura da imagem na qual estamos condicionando.

In [ ]:
# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()

In [ ]:
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 100

# Calculate output of image discriminator (PatchGAN)
patch = (1, opt.img_height // 2 ** 4, opt.img_width // 2 ** 4)

Instanciando o Gerador e Discriminador. No caso desse modelo, estamos usando a implementação definida no arquivo `models.py` presente no github que importamos. Ao importar esse módulo acima temos acesso à classe que instancia o objeto. A arqutietura do Gerador é uma UNet, já que essa tarefa tem um objetivo parecido com o de segmentação semântica. A arquitetura do Discriminador também é semelhante ao Discriminador padrão, com exceção de que ele da como saída um "mapa de escores", ao invés de um escore único. Essa técnica ficou conhecida como PatchGAN, porque em cada localidade desse mapa de escores, o Discriminador está avaliando um patch (corte) da imagem de entrada localmente, ao invés da imagem inteira.

In [ ]:
# Initialize generator and discriminator
generator = GeneratorUNet()
discriminator = Discriminator()

if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    criterion_GAN.cuda()
    criterion_pixelwise.cuda()

In [ ]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Definindo otimizadores:

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Definindo os objetos para carregar os dados:

In [ ]:
# Configure dataloaders
transforms_ = [
    transforms.Resize((opt.img_height, opt.img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ImageDataset("../../data/%s" % opt.dataset_name, transforms_=transforms_),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)

val_dataloader = DataLoader(
    ImageDataset("../../data/%s" % opt.dataset_name, transforms_=transforms_, mode="val"),
    batch_size=10,
    shuffle=True,
    num_workers=1,
)

In [ ]:
# Tensor type
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

Função para mostrar imagens geradas:

In [ ]:
def sample_images(batches_done):

    """Saves a generated sample from the validation set"""
    imgs = next(iter(val_dataloader))
    real_A = Variable(imgs["B"].type(Tensor))
    real_B = Variable(imgs["A"].type(Tensor))
    fake_B = generator(real_A)
    # img_sample = torch.cat((real_A.data, fake_B.data, real_B.data), -2)
    # save_image(img_sample, "images/%s/%s.png" % (opt.dataset_name, batches_done), nrow=5, normalize=True)
    
    fig, ax = plt.subplots(min(real_A.size(0), 2), 3, figsize=(12, 8))
    
    for i in range(min(real_A.size(0), 2)):
    
        ax[i, 0].imshow(real_A.data[i].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
        ax[i, 0].set_yticks([])
        ax[i, 0].set_xticks([])
        ax[i, 0].set_title('Real [A]')
        
        ax[i, 1].imshow(fake_B.data[i].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
        ax[i, 1].set_yticks([])
        ax[i, 1].set_xticks([])
        ax[i, 1].set_title('Fake [B]')
        
        ax[i, 2].imshow(real_B.data[i].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
        ax[i, 2].set_yticks([])
        ax[i, 2].set_xticks([])
        ax[i, 2].set_title('Real [B]')
    
    plt.show()

Definindo processo de treinamento:

In [ ]:
################################################################################
#  Training ####################################################################
################################################################################

prev_time = time.time()

for epoch in range(opt.epoch, opt.n_epochs + 1):
    
    for i, batch in enumerate(dataloader):

        # Model inputs
        real_A = Variable(batch["B"].type(Tensor))
        real_B = Variable(batch["A"].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *patch))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *patch))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        # Clearing gradients for G optimizer.
        optimizer_G.zero_grad()

        # GAN loss.
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, valid)
        
        # Pixel-wise loss
        loss_pixel = criterion_pixelwise(fake_B, real_B)

        # Total loss
        loss_G = loss_GAN + lambda_pixel * loss_pixel

        # G backward and optimizer step.
        loss_G.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Clearing gradients for D optimizer.
        optimizer_D.zero_grad()

        # Real loss
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, valid)

        # Fake loss
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, fake)

        # Total loss
        loss_D = 0.5 * (loss_real + loss_fake)

        # D backward and optimizer step.
        loss_D.backward()
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = opt.n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # If at sample interval save image
        if batches_done % opt.sample_interval == 0:

            # Print log
            sys.stdout.write(
                '[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, pixel: %f, adv: %f] ETA: %s'
                % (
                    epoch,
                    opt.n_epochs,
                    i,
                    len(dataloader),
                    loss_D.item(),
                    loss_G.item(),
                    loss_pixel.item(),
                    loss_GAN.item(),
                    time_left,
                )
            )
            
            sample_images(batches_done)

    if opt.checkpoint_interval != -1 and epoch % opt.checkpoint_interval == 0:
        
        # Save model checkpoints
        torch.save(generator.state_dict(), "saved_models/%s/generator_%d.pth" % (opt.dataset_name, epoch))
        torch.save(discriminator.state_dict(), "saved_models/%s/discriminator_%d.pth" % (opt.dataset_name, epoch))

# WGAN-GP

Uma outra modificação na GAN padrão que surgiu com o objetivo de melhorar a estabilidade do treinamento foi a [Wasserstein-GAN](https://arxiv.org/abs/1701.07875). Essa modificação consiste em alterar a função de perda da GAN. No caso da WGAN, a função de perda mede o quanto o discriminador consegue separar dados reais e falsos sem considerar um "limite" de separação. Enquanto o discriminador puder aumentar mais o "gap" entre o score para dados reais e falsos ele vai aumentar. A vantagem disso é que a sua derivada tem valores significativos mesmo quando o discriminador é muito bom em separar os dados. Portanto o gradiente para o Gerador sempre é informativo. 

Uma restrição descrita no artigo original da WGAN era que para que as vantagens teóricas que a função de perda de Wasserstein se concretizassem, o discriminador precisava ser uma função *1-Lipchitz contínua*. Isso significa que o gradiente do discriminador com relação à imagem de entrada deve ter norma menor que 1. Para assegurar essa restrição, o artigo original propunha *clipar* os pesos do discriminador em valores em um intervalo pequeno (como $[-0.01, 0.01]$).

Outro trabalho que extendeu o trabalho da WGAN propõe uma outra forma de assegurar essa restrição que funciona melhor: ter na função de perda um termo que penaliza a norma do gradiente de D ser diferente de 1:

![WGAN-GP loss](https://i.imgur.com/fjyTgFi.png)

Como essa função de perda proposta tem uma penalização para a norma do gradiente, esse trabalho ficou conhecido como [WGAN-GP](https://arxiv.org/abs/1704.00028) (*Wasserstein GAN Gradient Penalty*).

![WGAN-GP](https://github.com/eriklindernoren/PyTorch-GAN/raw/master/assets/wgan_gp.gif)

Importando bibliotecas necessárias:

In [ ]:
import os
import numpy as np
import math
import sys

import matplotlib.pyplot as plt
%matplotlib inline

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

Declarando hiperparâmetros usados para o treinamento:

In [ ]:
class Opt:
    n_epochs = 25 			## number of epochs of training
    batch_size = 64 		## size of the batches
    lr = 0.0002 			## adam: learning rate
    b1 = 0.5 				## adam: decay of first order momentum of gradient
    b2 = 0.999				## adam: decay of first order momentum of gradient
    n_cpu = 8				## number of cpu threads to use during batch generation
    latent_dim = 100		## dimensionality of the latent space
    img_size = 28			## size of each image dimension
    channels = 1			## number of image channels
    n_critic = 5			## number of training steps for discriminator per iter
    clip_value = 0.01		## lower and upper clip value for disc. weights
    sample_interval = 400	## interval betwen image samples

opt = Opt()

Definindo a arquitetura do Gerador e do Discriminador. A princípio, a arquitetura dos modelos pode ser constituída de camadas convolucionais (ou convolucional transposta) como definimos nos modelos anteriores. Porém, nesse experimento vamos usar camadas *fully connected* tanto para o gerador quanto para o discriminador. Mesmo assim, como a função de perda da WGAN-GP é mais poderosa, o gerador é capaz de aprender a criar imagens que se parecem com os dados originais do MNIST em poucas épocas.

In [ ]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [ ]:
# Loss weight for gradient penalty
lambda_gp = 10

Instanciando o gerador e o discriminador:

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

Criando os DataLoaders necessários para carregar os dados do MNIST:

In [ ]:
# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

Definindo os otimizadores:

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

Função para calcular o *gradient penalty*. Como o PyTorch já possui um módulo responsável pelos processamentos necessários para a diferenciação automática, podemos calcular essa penalização (e a sua derivada) através de funções do próprio [`autograd`](https://pytorch.org/docs/stable/autograd.html).

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Tensor(real_samples.shape[0], 1).fill_(1.0)
    fake.requires_grad_(False)
    
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

Procedimentos de treinamento:

In [ ]:
# ----------
#  Training
# ----------

batches_done = 0
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

        if batches_done % opt.sample_interval == 0:
            # save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
            
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

            nrow_, ncol_ = 2, 3
            c = 0
            fig, ax = plt.subplots(nrows=nrow_, ncols=ncol_, figsize=(8, 6))
            for i in range(nrow_):
                for j in range(ncol_):                    
                    ax[i, j].imshow(fake_imgs.data[c].detach().cpu().numpy().squeeze() * 0.5 + 0.5)
                    ax[i, j].set_yticks([])
                    ax[i, j].set_xticks([])
                    c += 1
            
            plt.show()

        batches_done += 1